Проект:

Запрос сформирован в связи с тем, что просмотр фильмов на языке оригинала является популярным и эффективным методом прокачки при изучении иностранных языков. При этом важно выбрать фильм, соответствующий уровню сложности студента, т.е. чтобы студент понимал 50-70% диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и определить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача:

Разработать ML-решение для автоматического определения уровня сложности англоязычных фильмов.

В решении поставленной задачи могут помочь следующие шаги:
1. Загрузить данные
2. Провести EDA
3. Попытаться собрать больше данных
4. изучить способы решения аналогичных задач: работа с текстовыми данными лемматизация, стемминг, стоп-слова и т.д., библиотеки NLP.
5. Построить базовую модель, оценить качество
6. Доработать решение (повторить шаги 3-6)

1. Понимание проблемы:
Тщательно изучите проблему, включая требования, ограничения и цели. Разбейте задачу на более мелкие подзадачи и убедитесь, что у вас есть четкое понимание того, что ожидается от решения.

2. Сбор и предварительная обработка данных:
Соберите набор данных транскриптов англоязычных фильмов с соответствующими уровнями сложности. Можно использовать такие источники, как субтитры, сценарии фильмов или онлайновые базы данных. Предварительно обработайте данные, очистив текст, удалив специальные символы и преобразовав их в строчные.

3. Эксплораторный анализ данных (EDA):
Проанализируйте набор данных, чтобы получить представление о распределении уровней сложности, длине текстов и любых закономерностях, которые могут возникнуть. Этот этап поможет лучше понять данные и направить процесс разработки функций.

4. Расширение данных:
Если набор данных ограничен, рассмотрите возможность его дополнения путем создания синтетических данных или использования таких методов, как обратный перевод, для создания дополнительных примеров. Это поможет повысить эффективность модели.

5. Предварительная обработка текста:
Применяются такие методы НЛП, как токенизация, лемматизация и удаление стоп-слов. Эти действия позволяют подготовить текстовые данные для ввода в модель.

6. Извлечение признаков с помощью трансформаторов:
Использование предварительно обученных моделей трансформаторов из библиотек, таких как Hugging Face Transformers. Эти модели, такие как BERT, GPT-2 или RoBERTa, были обучены на огромных объемах текста и могут извлекать из текстовых данных богатые контекстуальные вкрапления. Настройте эти модели на конкретную задачу, чтобы повысить их производительность.

7. Архитектура модели:
Разработайте архитектуру модели. Вы можете использовать PyTorch для создания собственных архитектур нейронных сетей или модифицировать существующие архитектуры для решения конкретной задачи. Входными данными для модели будут контекстные вкрапления, сгенерированные моделью трансформатора.

8. Базовая модель:
Постройте базовую модель, используя разработанную архитектуру. Обучите модель на своем наборе данных и оцените ее производительность с помощью соответствующих метрик для задачи классификации. Это послужит отправной точкой для итеративных улучшений.

9. Оценка и доработка модели:
Проанализируйте производительность базовой модели и определите области, требующие улучшения. Экспериментируйте с различными гиперпараметрами, архитектурой модели и методиками для повышения точности модели. Для оценки устойчивости модели можно использовать такие методы, как перекрестная валидация.

10. Итерация и оптимизация:
Итеративно дорабатывайте модель, возвращаясь к шагам 3-6. Собирайте больше данных, экспериментируйте с различными методиками и настраивайте модель для достижения лучших результатов.

11. Документирование и презентация:
Документируйте каждый этап работы, включая предварительную обработку данных, архитектуру модели, гиперпараметры и результаты. Создайте четкую и лаконичную презентацию с кратким описанием вашего подхода, возникших проблем и итоговой эффективности модели.

12. Будущие усовершенствования:
Рассмотрите дополнительные возможности и методы, которые могут еще больше повысить точность модели. Можно рассмотреть ансамблевые методы, более совершенные архитектуры трансформаторов или использование внешних ресурсов.

In [2]:
import os
import shutil
import re

import pandas as pd
import numpy as np

import torch

import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
movies_labels = pd.read_excel('data/English_scores/movies_labels.xlsx')
# movies_labels = pd.read_csv('data/English_scores/movies_labels.csv')

In [9]:
movies_labels

,id,movie,level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [103]:
# show films without subtitles in folder Subtitles
subtitles_folder = 'data/English_scores/Subtitles_all/Subtitles'

sub_names = os.listdir(subtitles_folder)
sub_names = [sub_name[:-4] for sub_name in sub_names]

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2
...,...,...
107,Venom(2018),B2
108,Warm_bodies(2013),B1
109,We_are_the_Millers(2013),B1
110,While_You_Were_Sleeping(1995),B1


In [115]:
movies_labels = movies_labels.loc[movies_labels['movie'].isin(sub_names)]

In [5]:
movies_labels.drop_duplicates(inplace=True)

In [118]:
movies_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   movie   108 non-null    object
 1   level   108 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


In [119]:
movies_labels.value_counts('level')

level
B1    43
A2    30
B2    29
C1     6
Name: count, dtype: int64

In [7]:
# change columns to lower
movies_labels.columns = movies_labels.columns.str.lower()

In [8]:
movies_labels.loc[movies_labels['level'] == 'A2/A2+', 'level'] = 'A2'
movies_labels.loc[movies_labels['level'] == 'B1, B2', 'level'] = 'B1'
movies_labels.loc[movies_labels['level'] == 'A2/A2+, B1', 'level'] = 'A2'

In [101]:
movies_labels.value_counts('level')

level
B2    101
B1     63
C1     40
A2     37
Name: count, dtype: int64

In [120]:
new_movies_labels = movies_labels.copy()

In [121]:
# add movies from folder (A1, A2, B1, B2, C1) in movies_labels and copy subtitles to folder
subtitles_folder = 'data/English_scores/Subtitles_all'
level_folders = ['A2', 'B1', 'B2', 'C1']

new_rows = []

for level_folder in level_folders:
    full_path = os.path.join(subtitles_folder, level_folder)
    for file in os.listdir(full_path):
        new_row = {'movie': file[:-4], 'level': level_folder}
        new_rows.append(new_row)

        shutil.copy(os.path.join(full_path, file), os.path.join(subtitles_folder, 'Subtitles', file))
        os.remove(os.path.join(full_path, file))

In [122]:
new_movies_df = pd.DataFrame(new_rows)
new_movies_labels = pd.concat([new_movies_labels, new_movies_df], ignore_index=True)

In [123]:
new_movies_df

,movie,level
0,Alice.In.Wonderland.2010.Bluray.1080p.DTSHD-MA...,A2
1,Charlie.and.the.Chocolate.Factory.2005.720p.Br...,A2
2,Chicken.Run.2000.1080p.BluRay.x264,A2
3,Despicable.Me.2.2013.720p.BluRay.x264-VeDeTT,A2
4,Dolphin.Tale.DVDRip.XviD-DiAMOND-CD1,A2
...,...,...
254,There.Will.Be.Blood[2007]DvDrip[Eng]-aXXo.en,C1
255,timpe-inception,C1
256,Trainspotting.1996.720p.BluRay.x264-SiNNERS.EN,C1
257,Twelve.Monkeys.(1995).DVDRip.XviD-UnSeeN,C1


In [124]:
new_movies_labels.value_counts('level')

level
B2    136
A2     81
B1     81
C1     69
Name: count, dtype: int64

In [125]:
# save new movies_labels
new_movies_labels.to_csv('data/English_scores/movies_labels_big.csv', index=False)

In [128]:
new_movies_labels

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2
...,...,...
362,There.Will.Be.Blood[2007]DvDrip[Eng]-aXXo.en,C1
363,timpe-inception,C1
364,Trainspotting.1996.720p.BluRay.x264-SiNNERS.EN,C1
365,Twelve.Monkeys.(1995).DVDRip.XviD-UnSeeN,C1


In [35]:
# add level in file name of subtitles
subtitles_folder = 'data/English_scores/Subtitles_all/Subtitles'

for file in os.listdir(subtitles_folder):
    try:
        level = movies_labels.loc[movies_labels['movie'] == file[:-4], 'level'].values[0]
        os.rename(os.path.join(subtitles_folder, file), os.path.join(subtitles_folder, f'{file[:-4]}_{level}.txt'))
    except IndexError:
        print(f'No level for {file}')

No level for .DS_Store
No level for 2001.A.Space.Odyssey.BluRay.720p.DTS.x264-CtrlHD.eng.srt
No level for A.Beautiful.Mind.2001.720p.HDTV.x264-S26.srt
No level for A.Clockwork.Orange.1971.1080p.BluRay.x264-TiMELORDS.srt
No level for Adaptation.2002.DVDRip.XviD-DiSSOLVE.srt
No level for Alice.In.Wonderland.2010.Bluray.1080p.DTSHD-MA.x264.dxva-FraMeSToR.ENG.srt
No level for Ameli.2001.x264.BDRip.720p.srt
No level for American.Psycho.2000.720p.BrRip.x264.YIFY.srt
No level for b-pd2wsa.srt
No level for b-pd2wsb.srt
No level for Back.To.The.Future.1985.720p.BluRay.x264.DTS-WiKi.eng.srt
No level for Black.Swan.2010.720p.BRRip.XviD.AC3-ViSiON.srt
No level for Blade.Runner[Remastered.Director's.Cut]DvDrip[Eng]-aXXo.srt
No level for Breaking_Bad_The_Movie(2017).srt
No level for Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix.srt
No level for Casper.srt
No level for Charlie.and.the.Chocolate.Factory.2005.720p.BrRip.x264.BOKUTOX.YIFY.srt
No level for Chicken.Run.2000.1080p

In [42]:
# change all files extensions in folder to .txt
# target_folder = 'data/English_scores/Subtitles_all/Subtitles'
#
# for file in os.listdir(target_folder):
#     if file.endswith('.srt'):
#         os.rename(os.path.join(target_folder, file), os.path.join(target_folder, file.replace('.srt', '.txt')))
#     elif file.endswith('.sub'):
#         os.rename(os.path.join(target_folder, file), os.path.join(target_folder, file.replace('.sub', '.txt')))
#     else:
#         print(f'File extension will not be changed: {file}')

File extension will not be changed: Aladdin.txt


In [51]:
# folder_path = 'data/English_scores/Subtitles_all/Subtitles'
#
# pattern = re.compile(r"[^a-zA-Z\s]")
#
# for filename in os.listdir(folder_path):
#     if filename.endswith(".txt"):
#         file_path = os.path.join(folder_path, filename)
#         with open(file_path, "r") as file:
#             content = file.read()
#
#         cleaned_content = re.sub(pattern, "", content)
#         cleaned_content = re.sub(r"\s+", " ", cleaned_content)
#         cleaned_content = cleaned_content.lower()
#
#         # Write the cleaned content back to the file
#         with open(file_path, "w") as file:
#             file.write(cleaned_content)


In [75]:
# create DataFrame with subtitles and levels using movies_labels and subtitles folder
subtitles_folder = 'data/English_scores/Subtitles_all/Subtitles'

subtitles = []
levels = []
film_names = []

for filename in os.listdir(subtitles_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(subtitles_folder, filename)
        with open(file_path, "r") as file:
            content = file.read()
            subtitles.append(content)
            film_names.append(filename[:-4])
            try:
                levels.append(movies_labels.loc[movies_labels['movie'] == filename[:-4], 'level'].values[0])
            except IndexError:
                print(f'No level for {filename}')
                levels.append(np.nan)


No level for Aladdin.txt
No level for Breaking_Bad_The_Movie(2017).txt
No level for Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix.txt
No level for Casper.txt
No level for Gogo_Loves_English.txt
No level for Harry_Potter_and_the_philosophers_stone(2001).txt
No level for Pride_and_Prejudice.txt
No level for The_Ghost_Writer.txt
No level for Up(2009).txt
No level for Westworld_scenes_of_Dr_Robert_Ford.txt


In [78]:
subtitles_df = pd.DataFrame({'film_name': film_names, 'subtitles': subtitles, 'level': levels})

In [103]:
subtitles_df

,film_name,subtitles,level
0,10_Cloverfield_lane(2016),font colorffffbfixed synced by bozxphd enjoy ...,B1
1,10_things_I_hate_about_you(1999),hey ill be right with you so cameron here you...,B1
2,A Goofy Movie,max max max roxanne whats wrong hello hello m...,A1
3,Aladdin(1992),ioh i come from a land from a faraway placei ...,A2
5,All_dogs_go_to_heaven(1989),captioning made possible by mgm home entertai...,A2
...,...,...,...
332,Westworld_scenes_of_Dr_Robert_Ford,music no ones complained music theres the lad...,C1
333,We_are_the_Millers(2013),ioh my godi iits fullon double rainbow all th...,B1
334,While_You_Were_Sleeping(1995),lucy iokay there are two things thati ii reme...,B1
335,z srt23 uk-bun Gullivers.Travels.1939.720p.Blu...,english bgullivers travels bbased on jonathan...,B2


In [104]:
subtitles_df.to_csv('data/English_scores/subtitles_df.csv', index=False)

In [ ]:
subtitles_df

In [62]:
movies_labels = pd.read_csv('data/English_scores/movies_labels.csv')


In [63]:
movies_labels

,movie,level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2
...,...,...
414,Suits.S03E06.720p.HDTV.x264-mSD.srt,C1
415,Suits.S03E07.HDTV.x264-mSD.srt,C1
416,Suits.S03E08.480p.HDTV.x264-mSD.srt,C1
417,Suits.S03E09.480p.HDTV.x264-mSD.srt,C1


In [64]:
sub_names = os.listdir('data/English_scores/Subtitles_all/Subtitles')

sub_filtr = set(sub_names) & set(movies_labels['movie'] + '.srt')
print(f'Количество фильмов, имеющих метку и субтитры: {len(sub_filtr)}')

Количество фильмов, имеющих метку и субтитры: 229


In [66]:
# фильмы которые не имеют метки
sub_filtr2 = set(sub_names) - set(movies_labels['movie'] + '.srt')
sub_filtr2

{'Breaking_Bad_The_Movie(2017).srt',
 'Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix.srt',
 'Casper.srt',
 'Crown, The S01E01 - Wolferton Splash.en.SDH.srt',
 'Crown, The S01E01 - Wolferton Splash.en.srt',
 'Crown, The S01E02 - Hyde Park Corner.en.SDH.srt',
 'Crown, The S01E02 - Hyde Park Corner.en.srt',
 'Crown, The S01E03 - Windsor.en.FORCED.srt',
 'Crown, The S01E03 - Windsor.en.SDH.srt',
 'Crown, The S01E03 - Windsor.en.srt',
 'Crown, The S01E04 - Act of God.en.SDH.srt',
 'Crown, The S01E04 - Act of God.en.srt',
 'Crown, The S01E05 - Smoke and Mirrors.en.FORCED.srt',
 'Crown, The S01E05 - Smoke and Mirrors.en.SDH.srt',
 'Crown, The S01E05 - Smoke and Mirrors.en.srt',
 'Crown, The S01E06 - Gelignite.en.SDH.srt',
 'Crown, The S01E06 - Gelignite.en.srt',
 'Crown, The S01E07 - Scientia Potentia Est.en.FORCED.srt',
 'Crown, The S01E07 - Scientia Potentia Est.en.SDH.srt',
 'Crown, The S01E07 - Scientia Potentia Est.en.srt',
 'Crown, The S01E08 - Pride & Joy.en.S

In [ ]:
import os
import zipfile

def extract_archives_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        if filename.endswith(".zip"):  # Предположим, что архивы имеют расширение .zip
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(folder_path)
            os.remove(filepath)  # Опционально: удалить исходный архив после разархивации

if __name__ == "__main__":
    target_folder = "data/zips"
    extract_archives_in_folder(target_folder)
    print("Разархивация завершена.")
